<a href="https://colab.research.google.com/github/tngus4334/LG-Aimers/blob/main/SUBMISSION_%EC%B5%9C%EC%A2%85%EC%A0%9C%EC%B6%9C%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline


import time
import warnings ; warnings.filterwarnings('ignore')
from tqdm import tqdm

from sklearn.preprocessing import Normalizer
from sklearn.multioutput import MultiOutputRegressor
from scipy.stats import norm

from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_absolute_error
import random
#import shap
#import optuna

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import ExtraTreeRegressor

import random
import os

import xgboost as xgb

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import BaggingRegressor

### 전처리

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/데이콘/자율주행센서/train.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘/자율주행센서/test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/자율주행센서/sample_submission.csv')

In [ ]:
train.drop(['X_02','X_04','X_10','X_11','X_23','X_47','X_48'],axis=1,inplace=True)
test.drop(['X_02','X_04','X_10','X_11','X_23','X_47','X_48'],axis=1,inplace=True)

### train 전처리

In [ ]:
# 스크류 삽입깊이 단차
train['max-min_X'] = train[['X_19','X_20','X_21','X_22']].max(axis=1) - train[['X_19','X_20','X_21','X_22']].min(axis=1)

In [ ]:
# 하우징 기울기
train['slope_X'] = train[['X_38','X_39','X_40']].max(axis = 1) - train[['X_38','X_39','X_40']].min(axis = 1)

In [ ]:
# 납량차이
train['SMT_X'] = train[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].max(axis = 1) - train[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].min(axis = 1)

In [ ]:
# 비슷한 종류들끼리 평균
train['X1~X6'] =  train[['X_01','X_06']].mean(axis=1)
train['X14~X18'] = train[['X_14','X_15','X_16','X_17','X_18']].mean(axis=1)
train['X19~X22'] = train[['X_19','X_20','X_21','X_22']].mean(axis=1)
train['X24~X29'] = train[['X_24','X_25','X_26','X_27','X_28','X_29']].mean(axis=1)
train['X30~X32'] = train[['X_30','X_32']].mean(axis=1)
train['X31~X33'] = train[['X_31','X_33']].mean(axis=1)
train['X38~X40'] = train[['X_38','X_39','X_40']].mean(axis=1)
train['X41~X45'] = train[['X_41','X_42','X_43','X_44','X_45']].mean(axis=1)
train['X50~X56'] = train[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].mean(axis=1)

In [ ]:
# 단차
train['con1_X'] = train['X_12'] - train['X_24']
train['con2_X'] = train['X_12'] - train['X_25']
train['con3_X'] = train['X_12'] - train['X_26']
train['con4_X'] = train['X_12'] - train['X_27']
train['con5_X'] = train['X_12'] - train['X_28']
train['con6_X'] = train['X_12'] - train['X_29']

In [ ]:
# 중복으로 넣기
train['nd_area_X1'] = train['X_08']*train['X_07']/train['X_03']
train['th_area_X1'] = train['X_09']*train['X_07']/train['X_03']

train['nd_area_X2'] = train['X_08']*train['X_07']/train['X_03']
train['th_area_X2'] = train['X_09']*train['X_07']/train['X_03']

In [ ]:
train = train.replace(train['X_24'].min(), train['X_24'].mean())

In [ ]:
# 무게 / 면적
train['X3/X7'] = train['X_03']/ train['X_07']

# 스크류 깊이 평균
train['X19X30'] = train[['X_19','X_30']].mean(axis=1)
train['X20X31'] = train[['X_20','X_31']].mean(axis=1)
train['X21X32'] = train[['X_21','X_32']].mean(axis=1)
train['X22X33'] = train[['X_22','X_33']].mean(axis=1)

In [ ]:
# 표준편차
train['X14~X18_std'] = train[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].std(axis=1)
train['X19~X22_std'] = train[['X_19','X_20','X_21','X_22']].std(axis=1)
train['X24~X29_std'] =  train[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].std(axis=1)

train['X38~X40_std'] = train[['X_38','X_39','X_40']].std(axis=1)

train['X41~X45_std'] = train[['X_41','X_42', 'X_43', 'X_44', 'X_45']].std(axis=1)
train['X50~X56_std'] = train[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].std(axis=1)

In [ ]:
# 중앙값
train['X14~X18_mid'] = train[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].median(axis=1)
train['X19~X22_mid'] = train[['X_19','X_20','X_21','X_22']].median(axis=1)
train['X24~X29_mid'] =  train[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].median(axis=1)

train['X38~X40_mid'] = train[['X_38','X_39','X_40']].median(axis=1)

train['X41~X45_mid'] = train[['X_41','X_42', 'X_43', 'X_44', 'X_45']].median(axis=1)
train['X50~X56_mid'] = train[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].median(axis=1)

In [ ]:
# 튀는값 평균으로 대치
mean_X_30 = train[(train['X_30']<1.7) & (train['X_30']>1)]['X_30'].mean()
mean_X_31 = train[(train['X_31']<5)]['X_31'].mean()
mean_X_32 = train[(train['X_32']<1.7) & (train['X_32']>1)]['X_32'].mean()
mean_X_33 = train[(train['X_31']<5)]['X_31'].mean()
mean_X_38 = train[(train['X_33']<0.7)]['X_33'].mean()

In [ ]:
train['X_30'] = train['X_30'].apply(lambda x : mean_X_30 if x >1.7 or x < 1 else x)
train['X_31'] = train['X_31'].apply(lambda x : mean_X_31 if x > 5 else x)
train['X_32'] = train['X_32'].apply(lambda x : mean_X_32 if x >1.7 or x < 1 else x)
train['X_33'] = train['X_33'].apply(lambda x : mean_X_33 if x > 5 else x)
train['X_38'] = train['X_38'].apply(lambda x : mean_X_38 if x > 0.7 else x)

In [ ]:
# 공차
train['gongcha_1_X'] = train[['X_38','X_39','X_40']].sum(axis = 1)
train['gongcha_2_X'] = train[['X_24','X_25','X_26','X_27','X_28','X_29']].sum(axis = 1)

In [ ]:
train['all_sum_X'] = train.sum(axis=1)
train['all_mean_X'] = train.mean(axis=1)

### test 전처리

In [ ]:
test['max-min_X'] = test[['X_19','X_20','X_21','X_22']].max(axis=1) - test[['X_19','X_20','X_21','X_22']].min(axis=1)

In [ ]:
# 하우징 기울기
test['slope_X'] = test[['X_38','X_39','X_40']].max(axis = 1) - test[['X_38','X_39','X_40']].min(axis = 1)

In [ ]:
# 납량차이
test['SMT_X'] = test[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].max(axis = 1) - test[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].min(axis = 1)

In [ ]:
test['X1~X6'] =  test[['X_01','X_06']].mean(axis=1)
test['X14~X18'] = test[['X_14','X_15','X_16','X_17','X_18']].mean(axis=1)
test['X19~X22'] = test[['X_19','X_20','X_21','X_22']].mean(axis=1)
test['X24~X29'] = test[['X_24','X_25','X_26','X_27','X_28','X_29']].mean(axis=1)
test['X30~X32'] = test[['X_30','X_32']].mean(axis=1)
test['X31~X33'] = test[['X_31','X_33']].mean(axis=1)
test['X38~X40'] = test[['X_38','X_39','X_40']].mean(axis=1)
test['X41~X45'] = test[['X_41','X_42','X_43','X_44','X_45']].mean(axis=1)
test['X50~X56'] = test[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].mean(axis=1)

In [ ]:
test['con1_X'] = test['X_12'] - test['X_24']
test['con2_X'] = test['X_12'] - test['X_25']
test['con3_X'] = test['X_12'] - test['X_26']
test['con4_X'] = test['X_12'] - test['X_27']
test['con5_X'] = test['X_12'] - test['X_28']
test['con6_X'] = test['X_12'] - test['X_29']

In [ ]:
test['nd_area_X1'] = test['X_08']*test['X_07']/test['X_03']
test['th_area_X1'] = test['X_09']*test['X_07']/test['X_03']

test['nd_area_X2'] = test['X_08']*test['X_07']/test['X_03']
test['th_area_X2'] = test['X_09']*test['X_07']/test['X_03']

In [ ]:
# 무게 / 면적
test['X3/X7'] = test['X_03']/ test['X_07']

# 스크류 깊이 평균
test['X19X30'] = test[['X_19','X_30']].mean(axis=1)
test['X20X31'] = test[['X_20','X_31']].mean(axis=1)
test['X21X32'] = test[['X_21','X_32']].mean(axis=1)
test['X22X33'] = test[['X_22','X_33']].mean(axis=1)

In [ ]:
# 표준편차
test['X14~X18_std'] = test[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].std(axis=1)
test['X19~X22_std'] = test[['X_19','X_20','X_21','X_22']].std(axis=1)
test['X24~X29_std'] =  test[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].std(axis=1)

test['X38~X40_std'] = test[['X_38','X_39','X_40']].std(axis=1)

test['X41~X45_std'] = test[['X_41','X_42', 'X_43', 'X_44', 'X_45']].std(axis=1)
test['X50~X56_std'] = test[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].std(axis=1)

In [ ]:
test['X14~X18_mid'] = test[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].median(axis=1)
test['X19~X22_mid'] = test[['X_19','X_20','X_21','X_22']].median(axis=1)
test['X24~X29_mid'] =  test[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].median(axis=1)

test['X38~X40_mid'] = test[['X_38','X_39','X_40']].median(axis=1)

test['X41~X45_mid'] = test[['X_41','X_42', 'X_43', 'X_44', 'X_45']].median(axis=1)
test['X50~X56_mid'] = test[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].median(axis=1)

In [ ]:
test['gongcha_1_X'] = test[['X_38','X_39','X_40']].sum(axis = 1)
test['gongcha_2_X'] = test[['X_24','X_25','X_26','X_27','X_28','X_29']].sum(axis = 1)

In [ ]:
test['all_sum_X'] = test.sum(axis=1)
test['all_mean_X'] = test.mean(axis=1)

In [ ]:
train_x = train.filter(regex='X') # Input : X Featrue
train_y = train.filter(regex='Y') # Output : Y Feature

test_x = test.drop(columns=['ID'])

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

(39607, 92)
(39607, 14)
(39608, 92)


In [ ]:
from sklearn.metrics import mean_squared_error

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

## Y_01

In [ ]:
lightgbm_Y_01 = lgb.LGBMRegressor(bagging_fraction=0.5110153478713623, bagging_freq=7,
              feature_fraction=0.44859576444673155,
              learning_rate=0.009309864945773093, min_child_samples=25,
              min_split_gain=0.27023270779753594, n_estimators=285,
              num_leaves=169, random_state=42, reg_alpha=1.0144064465268647e-08,
              reg_lambda=2.4670673097820773e-09).fit(train_x,train_y['Y_01'])

In [ ]:
xgb_Y_01 =  BaggingRegressor(base_estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                             colsample_bylevel=1,
                                             colsample_bynode=1,
                                             colsample_bytree=0.5116487103581219,
                                             gamma=0,
                                             importance_type='gain',
                                             interaction_constraints='',
                                             learning_rate=0.07977498757924376,
                                             max_delta_step=0, max_depth=3,
                                             min_child_weight=1,
                                             n_estimators=239, n_jobs=-1,
                                             num_parallel_tree=1,
                                             random_state=42,
                                             reg_alpha=1.296241788406927e-06,
                                             reg_lambda=2.1341766607100183,
                                             scale_pos_weight=31.042164955168403,
                                             subsample=0.7175531571705309,
                                             validate_parameters=1,
                                             verbosity=0), n_estimators=30, n_jobs=1, random_state=42).fit(train_x,train_y['Y_01'])

In [ ]:
gbr_Y_01 =GradientBoostingRegressor(learning_rate=0.012688323956682181, max_depth=9,
                          max_features=0.5317748828007712,
                          min_impurity_decrease=1.2229206693892366e-09,
                          min_samples_leaf=2, min_samples_split=4,
                          n_estimators=238, random_state=42,
                          subsample=0.916882961309737).fit(train_x,train_y['Y_01'])

In [ ]:
catboost_Y_01 = BaggingRegressor(base_estimator=CatBoostRegressor(depth= 8,
        l2_leaf_reg= 14,
        loss_function= 'RMSE',
        border_count= 254,
        verbose= False,
        random_strength= 0.5536883616570151,
        task_type= 'CPU',
        n_estimators= 208,
        random_state= 42,
        eta= 0.0474004231211728),n_estimators=30, n_jobs=1, random_state=42).fit(train_x,train_y['Y_01'])

In [ ]:
pred_gb_Y_01 = gbr_Y_01.predict(test_x)
#pred_xgb_Y_01 = xgb_Y_01.predict(test_x)
pred_lgb_Y_01 = lightgbm_Y_01.predict(test_x)
pred_cat_Y_01 = catboost_Y_01.predict(test_x)

In [ ]:
# pred_Y_01_ensemble = pred_lgb_Y_01

In [ ]:
pred_Y_01_ensemble =  pred_gb_Y_01*0.4 + pred_lgb_Y_01*0.5 + pred_cat_Y_01*0.1

In [ ]:
sample_submission['Y_01'] = pred_Y_01_ensemble

## Y_02

In [ ]:
lightgbm_Y_02 = lgb.LGBMRegressor(bagging_fraction=0.7642214017718282, bagging_freq=1,
              feature_fraction=0.8676065963799555,
              learning_rate=0.00864265864457987, min_child_samples=68,
              min_split_gain=0.2550392624761004, n_estimators=296,
              num_leaves=248, random_state=42, reg_alpha=2.306873241784726e-05,
              reg_lambda=0.07577327208137193).fit(train_x,train_y['Y_02'])

In [ ]:
# xgb_Y_02 =  BaggingRegressor(base_estimator=XGBRegressor(base_score=0.5, booster='gbtree',
#                                              colsample_bylevel=1,
#                                              colsample_bynode=1,
#                                              colsample_bytree=0.5116487103581219,
#                                              gamma=0, gpu_id=0,
#                                              importance_type='gain',
#                                              interaction_constraints='',
#                                              learning_rate=0.07977498757924376,
#                                              max_delta_step=0, max_depth=3,
#                                              min_child_weight=1,
#                                              n_estimators=239, n_jobs=-1,
#                                              num_parallel_tree=1,
#                                              random_state=42,
#                                              reg_alpha=1.296241788406927e-06,
#                                              reg_lambda=2.1341766607100237,
#                                              scale_pos_weight=31.042164955168403,
#                                              validate_parameters=1,
#                                              verbosity=0),n_estimators=30, n_jobs=1, random_state=42).fit(train_x,train_y['Y_02'])

In [ ]:
gbr_Y_02 =GradientBoostingRegressor(learning_rate=0.014211238225789829, max_depth=6,
                          max_features=0.45340193840915854,
                          min_impurity_decrease=6.173511671095757e-05,
                          min_samples_leaf=5, min_samples_split=9,
                          n_estimators=296, random_state=42,
                          subsample=0.7924572396498928).fit(train_x,train_y['Y_02'])

In [ ]:
catboost_Y_02 = BaggingRegressor(base_estimator=CatBoostRegressor(depth= 10,
 l2_leaf_reg= 1,
 loss_function= 'RMSE',
 border_count= 254,
 verbose= False,
 random_strength= 0.5448661113920873,
 task_type= 'CPU',
 n_estimators= 300,
 random_state= 42,
 eta= 0.019842002273507353),n_estimators=30, n_jobs=1, random_state=42).fit(train_x,train_y['Y_02'])

In [ ]:
pred_gb_Y_02 = gbr_Y_02.predict(test_x)
# pred_xgb_Y_02 = xgb_Y_02.predict(test_x)
pred_lgb_Y_02 = lightgbm_Y_02.predict(test_x)
pred_cat_Y_02 = catboost_Y_02.predict(test_x)

In [ ]:
pred_Y_02_ensemble =  pred_gb_Y_02*0.4 + pred_lgb_Y_02*0.5 + pred_cat_Y_02*0.1

In [ ]:
sample_submission['Y_02'] = pred_Y_02_ensemble

## Y_03

In [ ]:
lightgbm_Y_03 = lgb.LGBMRegressor(bagging_fraction=0.7220587788311053, bagging_freq=3,
              feature_fraction=0.8954078184343524,
              learning_rate=0.011976502687763995, min_child_samples=28,
              min_split_gain=0.16926539626922182, n_estimators=254,
              num_leaves=150, random_state=42, reg_alpha=5.096795970275427e-07,
              reg_lambda=2.052414799701436e-10).fit(train_x,train_y['Y_03'])

In [ ]:
# xgb_Y_03 = BaggingRegressor(base_estimator=XGBRegressor(base_score=0.5, booster='gbtree',
#                                              colsample_bylevel=1,
#                                              colsample_bynode=1,
#                                              colsample_bytree=0.8796040274792856,
#                                              gamma=0,
#                                              importance_type='gain',
#                                              interaction_constraints='',
#                                              learning_rate=0.042390822957329766,
#                                              max_delta_step=0, max_depth=3,
#                                              min_child_weight=4,                      
#                                              n_estimators=152, n_jobs=-1,
#                                              num_parallel_tree=1,
#                                              random_state=42,
#                                              reg_alpha=5.8000830531275704e-05,
#                                              reg_lambda=0.00023840848215456116,
#                                              scale_pos_weight=23.880181098699897,
#                                              subsample=0.6255071325761761,
#                                              tree_method='auto',
#                                              validate_parameters=1,
#                                              verbosity=0),n_estimators=30, n_jobs=1, random_state=42).fit(train_x,train_y['Y_03'])

In [ ]:
gbr_Y_03 =GradientBoostingRegressor(learning_rate=0.007920599935104354, max_depth=9,
                          max_features=0.9958042425603121,
                          min_impurity_decrease=1.3837142440602336e-05,
                          min_samples_leaf=3, min_samples_split=3,
                          n_estimators=291, random_state=42,
                          subsample=0.5769261150355292).fit(train_x,train_y['Y_03'])

In [ ]:
catboost_Y_03 = BaggingRegressor(base_estimator=CatBoostRegressor(depth= 7,
 l2_leaf_reg= 98,
 loss_function= 'RMSE',
 border_count= 254,
 verbose= False,
 random_strength= 0.7744738290468063,
 task_type= 'CPU',
 n_estimators= 271,
 random_state= 42,
 eta= 0.0831520545759692),n_estimators=30, n_jobs=1, random_state=42).fit(train_x,train_y['Y_03'])

In [ ]:
pred_gb_Y_03 = gbr_Y_03.predict(test_x)
# pred_xgb_Y_03 = xgb_Y_03.predict(test_x)
pred_lgb_Y_03 = lightgbm_Y_03.predict(test_x)
pred_cat_Y_03 = catboost_Y_03.predict(test_x)

In [ ]:
pred_Y_03_ensemble =  pred_gb_Y_03*0.4 + pred_lgb_Y_03*0.2 + pred_cat_Y_03*0.4

In [ ]:
sample_submission['Y_03'] = pred_Y_03_ensemble

## Y_04

In [ ]:
lightgbm_Y_04 = lgb.LGBMRegressor(bagging_fraction=0.9166500862884649, bagging_freq=1,
              feature_fraction=0.5335602774521525,
              learning_rate=0.016087809538596266, min_child_samples=69,
              min_split_gain=0.7959999534940877, n_estimators=294,
              num_leaves=177, random_state=42, reg_alpha=2.1751579363138357e-06,
              reg_lambda=5.824020093449812e-07).fit(train_x,train_y['Y_04'])

In [ ]:
xgb_Y_04 = BaggingRegressor(base_estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                             colsample_bylevel=1,
                                             colsample_bynode=1,
                                             colsample_bytree=0.9046805777392568,
                                             gamma=0, 
                                             importance_type='gain',
                                             
                                             learning_rate=0.04138204383280735,
                                             max_delta_step=0, max_depth=10,
                                             min_child_weight=4, 
                                             n_estimators=158, n_jobs=-1,
                                             num_parallel_tree=1,
                                             random_state=42,
                                             reg_alpha=3.2860883941242434e-05,
                                             reg_lambda=0.0604292172277978,
                                             scale_pos_weight=32.84823260811049,
                                             subsample=0.7615735018061627,
                                             tree_method='auto',
                                             validate_parameters=1,verbosity=0)).fit(train_x,train_y['Y_04'])

In [ ]:
gbr_Y_04 =GradientBoostingRegressor(learning_rate=0.010820435454957386, max_depth=11,
                          max_features=0.5970282733038268,
                          min_impurity_decrease=2.7262411803771476e-08,
                          min_samples_leaf=2, min_samples_split=7,
                          n_estimators=281, random_state=42,
                          subsample=0.8967129136197456).fit(train_x,train_y['Y_04'])

In [ ]:
rf_Y_04 = RandomForestRegressor(n_estimators = 1000, max_features = 'auto', 
                            min_samples_split = 50, bootstrap = True,
                            max_depth = 50, 
                            max_samples = 1,n_bins =128, n_streams = 2,
                            min_samples_leaf = 8).fit(train_x,train_y['Y_04'])

In [ ]:
catboost_Y_04 = BaggingRegressor(base_estimator=CatBoostRegressor(depth= 8,
 l2_leaf_reg= 2,
 loss_function= 'RMSE',
 border_count= 254,
 verbose= False,
 random_strength= 0.7788427100693812,
 task_type= 'CPU',
 n_estimators= 300,
 random_state= 42,
 eta= 0.053954589959171614),n_estimators=30, n_jobs=1, random_state=42).fit(train_x,train_y['Y_04'])

In [ ]:
#pred_gb_Y_04 = gbr_Y_04.predict(test_x)
pred_xgb_Y_04 = xgb_Y_04.predict(test_x)
pred_lgb_Y_04 = lightgbm_Y_04.predict(test_x)
#pred_cat_Y_04 = catboost_Y_04.predict(test_x)
#pred_rf_Y_04 = rf_Y_04.predict(test_x)

In [ ]:
pred_Y_04_ensemble =  pred_xgb_Y_04*0.2 + pred_lgb_Y_04*0.8

In [ ]:
sample_submission['Y_04'] = pred_Y_04_ensemble

## Y_05

In [ ]:
lightgbm_Y_05 = lgb.LGBMRegressor(bagging_fraction=0.9828668744908224, bagging_freq=0,
              feature_fraction=0.506810098674697,
              learning_rate=0.01367133250059816, min_child_samples=79,
              min_split_gain=0.26613342689378217, n_estimators=278,
              num_leaves=194, random_state=42, reg_alpha=0.002105994037805515,
              reg_lambda=0.6505283042879445).fit(train_x,train_y['Y_05'])

In [ ]:
xgb_Y_05 = BaggingRegressor(base_estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                             colsample_bylevel=1,
                                             colsample_bynode=1,
                                             colsample_bytree=0.8178246777102668,
                                             gamma=0,
                                             importance_type='gain',
                                             interaction_constraints='',
                                             learning_rate=0.031194386754750062,
                                             max_delta_step=0, max_depth=11,
                                             min_child_weight=1,
                                             n_estimators=220, n_jobs=-1,
                                             num_parallel_tree=1,
                                             random_state=42,
                                             reg_alpha=1.2826639855192837e-07,
                                             reg_lambda=0.00021082239281910143,
                                             scale_pos_weight=28.06019301111908,
                                             subsample=0.9041545496671812,
                                             tree_method='auto',
                                             validate_parameters=1,
                                             verbosity=0),n_estimators=30, random_state=42).fit(train_x,train_y['Y_05'])

In [ ]:
gbr_Y_05 = GradientBoostingRegressor(learning_rate=0.023003908615888507, max_depth=11,
                          max_features=0.5259419040666877,
                          min_impurity_decrease=2.066728470159484e-05,
                          min_samples_leaf=4, min_samples_split=8,
                          n_estimators=156, random_state=42,
                          subsample=0.7874162856084028).fit(train_x,train_y['Y_05'])

In [ ]:
catboost_Y_05 = BaggingRegressor(base_estimator=CatBoostRegressor(depth= 7,
                            l2_leaf_reg= 7,
                            loss_function= 'RMSE',
                            border_count= 254,
                            verbose= False,
                            random_strength= 0.493663486801853,
                            task_type= 'CPU',
                            n_estimators= 235,
                            random_state= 42,
                            eta= 0.09610669268285155),
                            bootstrap = True,
                            bootstrap_features = False,
                            max_features = 1.0,
                            max_samples = 1.0,
                            n_estimators = 30,
                            n_jobs = None,
                            oob_score = False,
                            random_state = 42,
                            verbose = 0,
                            warm_start = False).fit(train_x,train_y['Y_05'])

In [ ]:
rf_Y_05 =  RandomForestRegressor(n_estimators = 1000, max_features = 'auto', 
                            min_samples_split = 150, bootstrap = True,
                            max_depth = 50, 
                            max_samples = 1,
                            min_samples_leaf = 32).fit(train_x,train_y['Y_05'])

In [ ]:
# pred_gb_Y_05 = gbr_Y_05.predict(test_x)
pred_xgb_Y_05 = xgb_Y_05.predict(test_x)
pred_lgb_Y_05 = lightgbm_Y_05.predict(test_x)
pred_cat_Y_05 = catboost_Y_05.predict(test_x)
# pred_rf_Y_05 = rf_Y_05.predict(test_x)

In [ ]:
pred_Y_05_ensemble =  pred_xgb_Y_05*0.3 + pred_lgb_Y_05*0.5 + pred_cat_Y_05*0.2

In [ ]:
sample_submission['Y_05'] = pred_Y_05_ensemble

## Y_06

In [ ]:
lightgbm_Y_06 = lgb.LGBMRegressor(bagging_fraction=0.9579031283860148, bagging_freq=6,
              feature_fraction=0.6643860696661427,
              learning_rate=0.020142699080470918, min_child_samples=16,
              min_split_gain=0.7155186457137692, n_estimators=270,
              num_leaves=228, random_state=42, reg_alpha=0.3433202364550086,
              reg_lambda=0.8027623126620722).fit(train_x,train_y['Y_06'])

In [ ]:
xgb_Y_06 = BaggingRegressor(base_estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                             colsample_bylevel=1,
                                             colsample_bynode=1,
                                             colsample_bytree=0.9046805777392568,
                                             gamma=0, gpu_id=0,
                                             importance_type='gain',
                                             interaction_constraints='',
                                             learning_rate=0.04138204383280735,
                                             max_delta_step=0, max_depth=10,
                                             min_child_weight=4,
                                             n_estimators=158, n_jobs=-1,
                                             num_parallel_tree=1,
                                             random_state=42,
                                             reg_alpha=3.2860883941242434e-05,
                                             reg_lambda=0.0604292172277978,
                                             scale_pos_weight=32.84823260811049,
                                             subsample=0.7615735018061627,
                                             validate_parameters=1,
                                             verbosity=0)).fit(train_x,train_y['Y_06'])

In [ ]:
gbr_Y_06 = GradientBoostingRegressor(learning_rate=0.04549583802747526, max_depth=9,
                          max_features=0.4667421108576381,
                          min_impurity_decrease=6.098855968050519e-06,
                          min_samples_leaf=4, min_samples_split=3,
                          random_state=42, subsample=0.9310311336710109).fit(train_x,train_y['Y_06'])

In [ ]:
catboost_Y_06 = CatBoostRegressor(eval_metric='RMSE',iterations= 277,sampling_frequency = 'PerTree',
                        leaf_estimation_method= 'Newton',grow_policy= 'SymmetricTree',penalties_coefficient= 1,
                        boosting_type = 'Plain', model_shrink_mode = 'Constant',feature_border_type= 'GreedyLogSum', l2_leaf_reg= 31, random_strength= 0.3085096776485443, rsm= 1,
                        boost_from_average = True, model_size_reg =  0.5, subsample = 0.800000011920929, use_best_model= False,
                        random_seed = 42,depth= 10, posterior_sampling= False, border_count= 254, 
                        learning_rate= 0.3210740387439728,max_leaves = 1024).fit(train_x,train_y['Y_06'])

0:	learn: 1.8371796	total: 207ms	remaining: 57s
1:	learn: 1.7914204	total: 397ms	remaining: 54.6s
2:	learn: 1.7555192	total: 600ms	remaining: 54.8s
3:	learn: 1.7219595	total: 799ms	remaining: 54.6s
4:	learn: 1.6961037	total: 1.04s	remaining: 56.9s
5:	learn: 1.6705781	total: 1.25s	remaining: 56.2s
6:	learn: 1.6325192	total: 1.44s	remaining: 55.4s
7:	learn: 1.6236955	total: 1.64s	remaining: 55.2s
8:	learn: 1.6148365	total: 1.83s	remaining: 54.6s
9:	learn: 1.5887219	total: 2.05s	remaining: 54.6s
10:	learn: 1.5686639	total: 2.25s	remaining: 54.5s
11:	learn: 1.5442961	total: 2.45s	remaining: 54.1s
12:	learn: 1.5363183	total: 2.65s	remaining: 53.8s
13:	learn: 1.5274764	total: 2.84s	remaining: 53.4s
14:	learn: 1.5106381	total: 3.06s	remaining: 53.5s
15:	learn: 1.5102167	total: 3.25s	remaining: 53s
16:	learn: 1.5097079	total: 3.44s	remaining: 52.7s
17:	learn: 1.5092994	total: 3.64s	remaining: 52.4s
18:	learn: 1.5090009	total: 3.84s	remaining: 52.1s
19:	learn: 1.5087035	total: 3.9s	remaining: 5

In [ ]:
# pred_gb_Y_06 = gbr_Y_06.predict(test_x)
# pred_xgb_Y_06 = xgb_Y_06.predict(test_x)
pred_lgb_Y_06 = lightgbm_Y_06.predict(test_x)
pred_cat_Y_06 = catboost_Y_06.predict(test_x)

In [ ]:
pred_Y_06_ensemble =  pred_lgb_Y_06*0.4 + pred_cat_Y_06*0.6

In [ ]:
sample_submission['Y_06'] = pred_Y_06_ensemble

## Y_07

In [ ]:
lightgbm_Y_07 = lgb.LGBMRegressor(bagging_fraction=0.838287407091275, bagging_freq=0,
              feature_fraction=0.8676948933076764,
              learning_rate=0.01545432563662372, min_child_samples=33,
              min_split_gain=0.13116484702947007, n_estimators=233,
              num_leaves=181, random_state=42, reg_alpha=2.42986213512713e-05,
              reg_lambda=1.7260007776053446e-05).fit(train_x,train_y['Y_07'])

In [ ]:
xgb_Y_07 = BaggingRegressor(base_estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                             colsample_bylevel=1,
                                             colsample_bynode=1,
                                             colsample_bytree=0.9090073829612466,
                                             gamma=0, 
                                             importance_type='gain',
                                             interaction_constraints='',
                                             learning_rate=0.080404022381391,
                                             max_delta_step=0, max_depth=1,
                                             min_child_weight=3,
                                             n_estimators=131, n_jobs=-1,
                                             num_parallel_tree=1,
                                             random_state=42,
                                             reg_alpha=2.774507576975746e-08,
                                             reg_lambda=2.082183691907341e-09,
                                             scale_pos_weight=17.54314339877777,
                                             subsample=0.9543277631300153,
                                             tree_method='auto',
                                             validate_parameters=1,
                                             verbosity=0),n_estimators=30, random_state=42).fit(train_x,train_y['Y_07'])

In [ ]:
gbr_Y_07 = GradientBoostingRegressor(learning_rate=0.01025071605101497, max_depth=11,
                          max_features=0.9584256172822183,
                          min_impurity_decrease=1.7165270372995716e-09,
                          min_samples_leaf=5, min_samples_split=9,
                          n_estimators=251, random_state=42,
                          subsample=0.5868763275242731).fit(train_x,train_y['Y_07'])

In [ ]:
catboost_Y_07 = BaggingRegressor(base_estimator=CatBoostRegressor(depth= 10,
 l2_leaf_reg= 102,
 loss_function= 'RMSE',
 border_count= 254,
 verbose= False,
 random_strength= 0.5381601680403292,
 n_estimators= 254,
 random_state= 42,
 eta= 0.1252529497263127),
 bootstrap= True,
 bootstrap_features= False,
 max_features= 1.0,
 max_samples= 1.0,
 n_estimators= 30,
 n_jobs= 1,
 oob_score= False,
 random_state= 42,
 verbose= 0,
 warm_start= False).fit(train_x,train_y['Y_07'])

In [ ]:
pred_gb_Y_07 = gbr_Y_07.predict(test_x)
#pred_xgb_Y_07 = xgb_Y_07.predict(test_x)
pred_lgb_Y_07 = lightgbm_Y_07.predict(test_x)
#pred_cat_Y_07 = catboost_Y_07.predict(test_x)

In [ ]:
pred_Y_07_ensemble =  pred_gb_Y_07*0.3 + pred_lgb_Y_07*0.7

In [ ]:
sample_submission['Y_07'] = pred_Y_07_ensemble

## Y_08

In [ ]:
lightgbm_Y_08 = lgb. LGBMRegressor(bagging_fraction=0.9699083434585961, bagging_freq=6,
              feature_fraction=0.4962524707401136,
              learning_rate=0.019402617789498314, min_child_samples=30,
              min_split_gain=0.2987415736694793, n_estimators=300,
              num_leaves=151, random_state=42, reg_alpha=1.5213394690434816,
              reg_lambda=0.00013304046059925623).fit(train_x,train_y['Y_08'])

In [ ]:
xgb_Y_08 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7616586588644908, gamma=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.030051429007604148, max_delta_step=0, max_depth=4,
             min_child_weight=4,
             n_estimators=289, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=1.1746549401580202e-10,
             reg_lambda=3.4997739011325505e-07,
             scale_pos_weight=28.339652964857382, subsample=0.7199764850350032,
             tree_method='auto', validate_parameters=1, verbosity=0).fit(train_x,train_y['Y_08'])

In [ ]:
gbr_Y_08 = GradientBoostingRegressor(learning_rate=0.03001726813934206, max_depth=11,
                          max_features=0.43741151687761776,
                          min_impurity_decrease=0.1308799784233315,
                          min_samples_leaf=3, min_samples_split=6,
                          random_state=42, subsample=0.8082785040960712).fit(train_x,train_y['Y_08'])

In [ ]:
catboost_Y_08 = CatBoostRegressor(depth= 9, l2_leaf_reg= 132,loss_function='RMSE',border_count= 254, verbose = False,
random_strength = 0.047639310090865764, task_type='CPU', n_estimators= 111, random_state= 42,eta= 0.19044602883699066).fit(train_x,train_y['Y_08'])

In [ ]:
rf_Y_08 = RandomForestRegressor(n_estimators = 1000, max_features = 'auto', 
                            min_samples_split = 100, bootstrap = True,
                            max_depth = 20, 
                            max_samples = 1,
                            min_samples_leaf = 16).fit(train_x,train_y['Y_08'])

In [ ]:
pred_gb_Y_08 = gbr_Y_08.predict(test_x)
#pred_xgb_Y_08 = xgb_Y_08.predict(test_x)
pred_lgb_Y_08 = lightgbm_Y_08.predict(test_x)
#pred_cat_Y_08 = catboost_Y_08.predict(test_x)
pred_rf_Y_08 = rf_Y_08.predict(test_x)

In [ ]:
pred_Y_08_ensemble = pred_gb_Y_08*0.2 + pred_lgb_Y_08*0.6 + pred_rf_Y_08*0.2

In [ ]:
sample_submission['Y_08'] = pred_Y_08_ensemble

## Y_09

In [ ]:
lightgbm_Y_09 = lgb.LGBMRegressor(bagging_fraction=0.847529712693275, bagging_freq=3,
              feature_fraction=0.4837674943957988,
              learning_rate=0.011288169616087126, min_child_samples=47,
              min_split_gain=0.7969668112627418, n_estimators=297,
              num_leaves=252, random_state=42, reg_alpha=3.4116973247692774e-05,
              reg_lambda=0.0006068803104015331).fit(train_x,train_y['Y_09'])

In [ ]:
xgb_Y_09 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8386343579954545, gamma=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05230531673933179, max_delta_step=0, max_depth=8,
             min_child_weight=4,
             n_estimators=118, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=3.9707140325892785, reg_lambda=1.3148500136982597,
             scale_pos_weight=44.76691454427343, subsample=0.3119106694940679,
             tree_method='auto', validate_parameters=1, verbosity=0).fit(train_x,train_y['Y_09'])

In [ ]:
gbr_Y_09 = GradientBoostingRegressor(learning_rate=0.013140069670011975, max_depth=9,
                          max_features=0.6818838242801941,
                          min_impurity_decrease=8.670784876694156e-08,
                          min_samples_leaf=2, min_samples_split=3,
                          n_estimators=272, random_state=42,
                          subsample=0.5480888678452744).fit(train_x,train_y['Y_09'])

In [ ]:
catboost_Y_09 = CatBoostRegressor(nan_mode= 'Min',
 eval_metric= 'RMSE',
 iterations= 1000,
 sampling_frequency= 'PerTree',
 leaf_estimation_method= 'Newton',
 grow_policy ='SymmetricTree',
 penalties_coefficient= 1,
 boosting_type= 'Plain',
 model_shrink_mode= 'Constant',
 feature_border_type= 'GreedyLogSum',
 l2_leaf_reg= 3,
 random_strength= 1,
 rsm= 1,
 boost_from_average= True,
 model_size_reg= 0.5,
 subsample= 0.800000011920929,
 use_best_model= False,
 random_seed= 42,
 depth= 6,
 posterior_sampling= False,
 border_count= 254,
 sparse_features_conflict_fraction= 0,
 leaf_estimation_backtracking= 'AnyImprovement',
 best_model_min_trees= 1,
 model_shrink_rate= 0,
 min_data_in_leaf= 1,
 loss_function= 'RMSE',
 learning_rate= 0.06920699775218964,
 score_function= 'Cosine',
 task_type= 'CPU',
 leaf_estimation_iterations= 1,
 bootstrap_type= 'MVS',
 max_leaves= 64).fit(train_x,train_y['Y_09'])

In [ ]:
rf_Y_09 = RandomForestRegressor(n_estimators = 1000, max_features = 'auto', 
                            min_samples_split = 100, bootstrap = True,
                            max_depth = 20, 
                            max_samples = 1,n_bins =256, n_streams = 8,
                            min_samples_leaf = 20).fit(train_x,train_y['Y_09'])

In [ ]:
pred_gb_Y_09 = gbr_Y_09.predict(test_x)
# pred_xgb_Y_09 = xgb_Y_09.predict(test_x)
pred_lgb_Y_09 = lightgbm_Y_09.predict(test_x)
# pred_cat_Y_09 = catboost_Y_09.predict(test_x)
# pred_rf_Y_09 = rf_Y_09.predict(test_x)

In [ ]:
pred_Y_09_ensemble =  pred_gb_Y_09*0.1 + pred_lgb_Y_09*0.9

In [ ]:
sample_submission['Y_09'] = pred_Y_09_ensemble

## Y_10

In [ ]:
lightgbm_Y_10 = lgb.LGBMRegressor(bagging_fraction=0.4, bagging_freq=0, feature_fraction=0.4,							
              learning_rate=0.016937232133213884, min_child_samples=25,							
              n_estimators=300, num_leaves=148, random_state=42,							
              reg_alpha=5.405882409074974e-07, reg_lambda=10.0).fit(train_x,train_y['Y_10'])

In [ ]:
xgb_Y_10 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.567297632104587, gamma=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.03286530816784422, max_delta_step=0, max_depth=11,
             min_child_weight=4,
             n_estimators=168, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=1.4997413758623726e-08, reg_lambda=1e-10,
             scale_pos_weight=1.0, subsample=1.0, tree_method='auto',
             validate_parameters=1, verbosity=0).fit(train_x,train_y['Y_10'])

In [ ]:
gbr_Y_10 =  GradientBoostingRegressor(learning_rate=0.03728846132297749, max_depth=8,
                          max_features=0.5835554838846452,
                          min_impurity_decrease=0.0003259794045036959,
                          min_samples_leaf=4, min_samples_split=10,
                          random_state=42, subsample=0.9592912790914383).fit(train_x,train_y['Y_10'])

In [ ]:
catboost_Y_10 = CatBoostRegressor(loss_function= 'RMSE',
 border_count= 254,
 verbose= False,
 task_type= 'CPU',
 random_state= 42).fit(train_x,train_y['Y_10'])

In [ ]:
rf_Y_10 = RandomForestRegressor(n_estimators = 1000, max_features = 'auto', 
                            min_samples_split = 20, bootstrap = True,
                            max_depth = 16, 
                            max_samples = 1,n_bins =64, n_streams = 4,
                            min_samples_leaf = 16).fit(train_x,train_y['Y_10'])

In [ ]:
# pred_gb_Y_10 = gbr_Y_10.predict(test_x)
pred_xgb_Y_10 = xgb_Y_10.predict(test_x)
pred_lgb_Y_10 = lightgbm_Y_10.predict(test_x)
pred_cat_Y_10 = catboost_Y_10.predict(test_x)
#pred_rf_Y_10 = rf_Y_10.predict(test_x)

In [ ]:
pred_Y_10_ensemble =  pred_xgb_Y_10*0.2 + pred_lgb_Y_10*0.7 + pred_cat_Y_10*0.1

In [ ]:
sample_submission['Y_10'] = pred_Y_10_ensemble

## Y_11

In [ ]:
lightgbm_Y_11 = LGBMRegressor(bagging_fraction=0.8113283298264667, bagging_freq=3,
              feature_fraction=0.4756162193937983,
              learning_rate=0.02649749254706113, min_child_samples=55,
              min_split_gain=0.21623426116682082, n_estimators=168,
              num_leaves=229, random_state=42, reg_alpha=0.0002061574574317737,
              reg_lambda=2.9345383986078057).fit(train_x,train_y['Y_11'])

In [ ]:
xgb_Y_11 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5862938672993847, gamma=0,
             gpu_id=0, importance_type='gain', interaction_constraints='',
             learning_rate=0.08642088399402323, max_delta_step=0, max_depth=3,
             min_child_weight=1,
             n_estimators=232, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=4.1088049949702336e-07, reg_lambda=9.091981906252093,
             scale_pos_weight=48.88288898839123, subsample=0.6687637979858317,
            validate_parameters=1, verbosity=0).fit(train_x,train_y['Y_11'])

In [ ]:
gbr_Y_11 = GradientBoostingRegressor(learning_rate=0.011566390954303105, max_depth=11,
                          max_features=0.9041549708966354,
                          min_impurity_decrease=9.415107773747543e-06,
                          min_samples_leaf=4, n_estimators=206, random_state=42,
                          subsample=0.6049198426048275).fit(train_x,train_y['Y_11'])

In [ ]:
catboost_Y_11 = BaggingRegressor(base_estimator= CatBoostRegressor(depth =  7,
 l2_leaf_reg =  9,
 loss_function =  'RMSE',
 border_count =  254,
 verbose =  False,
 random_strength =  0.23028936602081082,
 task_type =  'CPU',
 n_estimators =  186,
 random_state =  42,
 eta =  0.10548240014937499),
 bootstrap = True,
 bootstrap_features = False,
 max_features = 1.0,
 max_samples = 1.0,
 n_jobs = 1,
 oob_score = False,
 random_state = 42,
 verbose = 0,
 n_estimators = 30,
 warm_start = False).fit(train_x,train_y['Y_11'])

In [ ]:
rf_Y_11 = RandomForestRegressor(n_estimators = 1000, max_features = 'auto', 
                            min_samples_split = 100, bootstrap = True,
                            max_depth = 20, 
                            max_samples = 1,n_bins =128, n_streams = 4,
                            min_samples_leaf = 25).fit(train_x,train_y['Y_11'])

In [ ]:
pred_gb_Y_11 = gbr_Y_11.predict(test_x)
# pred_xgb_Y_11 = xgb_Y_11.predict(test_x)
pred_lgb_Y_11 = lightgbm_Y_11.predict(test_x)
# pred_cat_Y_11 = catboost_Y_11.predict(test_x)
pred_rf_Y_11 = rf_Y_11.predict(test_x)

In [ ]:
pred_Y_11_ensemble =  pred_gb_Y_11*0.1 + pred_lgb_Y_11*0.8 + pred_rf_Y_11*0.1

In [ ]:
sample_submission['Y_11'] = pred_Y_11_ensemble

## Y_12

In [ ]:
lightgbm_Y_12 = lgb.LGBMRegressor(bagging_fraction=0.7308876198885925, bagging_freq=0,						
              feature_fraction=0.5343591110601751,						
              learning_rate=0.021772535138333004, min_child_samples=49,						
              min_split_gain=0.4979301715911596, n_estimators=250,						
              num_leaves=215, random_state=42, reg_alpha=0.00039646806445988895,						
              reg_lambda=0.0003866970843679147)	.fit(train_x,train_y['Y_12'])

In [ ]:
xgb_Y_12 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5862938672993847, gamma=0,
            importance_type='gain', interaction_constraints='',
             learning_rate=0.08642088399402323, max_delta_step=0, max_depth=3,
             min_child_weight=1,
             n_estimators=232, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=4.1088049949702336e-07, reg_lambda=9.091981906252093,
             scale_pos_weight=48.88288898839123, subsample=0.6687637979858317,
              validate_parameters=1, verbosity=0).fit(train_x,train_y['Y_12'])

In [ ]:
gbr_Y_12 = GradientBoostingRegressor(learning_rate=0.03854796434033087, max_depth=9,
                          max_features=0.8373237445893021,
                          min_impurity_decrease=0.44778865969295006,
                          min_samples_leaf=2, min_samples_split=7,
                          random_state=42, subsample=0.7519692756498256).fit(train_x,train_y['Y_12'])

In [ ]:
catboost_Y_12 =CatBoostRegressor(depth= 11,
 l2_leaf_reg= 1,
 loss_function= 'RMSE',
 border_count= 254,
 verbose= False,
 random_strength= 0.12088734224551677,
 task_type= 'CPU',
 n_estimators= 300,
 random_state= 42,
 eta= 0.022541488999505567).fit(train_x,train_y['Y_12'])

In [ ]:
rf_Y_12 = RandomForestRegressor(n_estimators = 1000, max_features = 'auto', 
                            min_samples_split = 50, bootstrap = True,
                            max_depth = 30, 
                            max_samples = 1,n_bins =128, n_streams = 8,
                            min_samples_leaf = 16).fit(train_x,train_y['Y_12'])

In [ ]:
pred_gb_Y_12 = gbr_Y_12.predict(test_x)
#pred_xgb_Y_12 = xgb_Y_12.predict(test_x)
pred_lgb_Y_12 = lightgbm_Y_12.predict(test_x)
# pred_cat_Y_12 = catboost_Y_12.predict(test_x)
pred_rf_Y_12 = rf_Y_12.predict(test_x)

In [ ]:
pred_Y_12_ensemble =  pred_gb_Y_12*0.4 +  pred_lgb_Y_12*0.3 + pred_rf_Y_12*0.3

In [ ]:
sample_submission['Y_12'] = pred_Y_12_ensemble

## Y_13

In [ ]:
lightgbm_Y_13 = lgb.LGBMRegressor(bagging_fraction=0.562177058115405, bagging_freq=0,
              feature_fraction=0.9023546637887719,
              learning_rate=0.01637180684883515, min_child_samples=52,
              min_split_gain=0.5525322291188884, n_estimators=234,
              num_leaves=196, random_state=42, reg_alpha=2.1036670016919905e-10,
              reg_lambda=2.9600855173354055).fit(train_x,train_y['Y_13'])

In [ ]:
xgb_Y_13 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7781793006393881, gamma=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.11079581903277634, max_delta_step=0, max_depth=3,
             min_child_weight=2,
             n_estimators=214, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=1.9612752633971965e-06, reg_lambda=0.005923918160674718,
             scale_pos_weight=1.830823844017683, subsample=0.8587829049947746,
             tree_method='auto', validate_parameters=1, verbosity=0).fit(train_x,train_y['Y_13'])

In [ ]:
gbr_Y_13 = GradientBoostingRegressor(learning_rate=0.03854796434033087, max_depth=9,
                          max_features=0.8373237445893021,
                          min_impurity_decrease=0.44778865969295006,
                          min_samples_leaf=2, min_samples_split=7,
                          random_state=42, subsample=0.7519692756498256).fit(train_x,train_y['Y_13'])

In [ ]:
catboost_Y_13 = CatBoostRegressor(depth = 11,
 l2_leaf_reg = 2,
 loss_function = 'RMSE',
 border_count = 254,
 verbose = False,
 random_strength = 0.08437286101356153,
 task_type = 'CPU',
 n_estimators = 300,
 random_state = 42,
 eta = 0.03180487549163332).fit(train_x,train_y['Y_13'])

In [ ]:
rf_Y_13 = RandomForestRegressor(n_estimators = 1000, max_features = 'auto', 
                            min_samples_split = 100, bootstrap = True,
                            max_depth = 20, 
                            max_samples = 1,n_bins =128, n_streams = 2,
                            min_samples_leaf = 20).fit(train_x,train_y['Y_13'])

In [ ]:
pred_gb_Y_13 = gbr_Y_13.predict(test_x)
# pred_xgb_Y_13 = xgb_Y_13.predict(test_x)
pred_lgb_Y_13 = lightgbm_Y_13.predict(test_x)
pred_cat_Y_13 = catboost_Y_13.predict(test_x)
pred_rf_Y_13 = rf_Y_13.predict(test_x)

In [ ]:
pred_Y_13_ensemble = pred_gb_Y_13*0.1 +pred_lgb_Y_13*0.5 + pred_cat_Y_13*0.2 + pred_rf_Y_13*0.2

In [ ]:
sample_submission['Y_13'] = pred_Y_13_ensemble

## Y_14

In [ ]:
lightgbm_Y_14 = lgb.LGBMRegressor(bagging_fraction=0.562177058115405, bagging_freq=0,
              feature_fraction=0.9023546637887719,
              learning_rate=0.01637180684883515, min_child_samples=52,
              min_split_gain=0.5525322291188884, n_estimators=234,
              num_leaves=196, random_state=42, reg_alpha=2.1036670016919905e-10,
              reg_lambda=2.9600855173354055).fit(train_x,train_y['Y_14'])

In [ ]:
xgb_Y_14 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6294674641087924, gamma=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.08974284583794521, max_delta_step=0, max_depth=4,
             min_child_weight=2,
             n_estimators=155, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=3.5772679768707163, reg_lambda=1.8484658552802888e-10,
             scale_pos_weight=49.04172498933266, subsample=0.907050064431688).fit(train_x,train_y['Y_14'])

[12:05:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
gbr_Y_14 = GradientBoostingRegressor(learning_rate=0.03055416183446505, max_depth=11,
                          max_features=0.9271370796951685,
                          min_impurity_decrease=1.7601560220672312e-08,
                          min_samples_leaf=3, min_samples_split=6,
                          n_estimators=155, random_state=42,
                          subsample=0.811344094127121).fit(train_x,train_y['Y_14'])

In [ ]:
catboost_Y_14 = CatBoostRegressor(depth = 10,l2_leaf_reg = 147,
 loss_function = 'RMSE',
 border_count = 254,
 verbose = False,
 random_strength = 0.17872556513557733,
 task_type = 'CPU',
 n_estimators = 282,
 random_state = 42,
 eta = 0.17036206261250125).fit(train_x,train_y['Y_14'])

In [ ]:
rf_Y_14 = RandomForestRegressor(n_estimators = 1000, max_features = 'auto', 
                            min_samples_split = 100, bootstrap = True,
                            max_depth = 20, 
                            max_samples = 1,
                            min_samples_leaf = 25).fit(train_x,train_y['Y_14'])

In [ ]:
pred_gb_Y_14 = gbr_Y_14.predict(test_x)
# pred_xgb_Y_14 = xgb_Y_14.predict(test_x)
pred_lgb_Y_14 = lightgbm_Y_14.predict(test_x)
pred_cat_Y_14 = catboost_Y_14.predict(test_x)
pred_rf_Y_14 = rf_Y_14.predict(test_x)

In [ ]:
pred_Y_14_ensemble =  pred_gb_Y_14*0.3 +pred_lgb_Y_14*0.5 + pred_cat_Y_14*0.1 + pred_rf_Y_14*0.1

In [ ]:
sample_submission['Y_14'] = pred_Y_14_ensemble

## 제출

In [ ]:
sample_submission.to_csv('submission_0823.csv', index = False)

In [ ]:
pd.read_csv('submission_0823.csv')